In [22]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
# from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import MobileNet, MobileNetV2, VGG16, VGG19, Xception,ResNet101V2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras import datasets, layers, Model, Sequential, losses
# 建立類別名稱轉class idx字典
aa = pd.read_csv("/kaggle/input/face-recognition-xt122/ClassMap.csv")
a = aa.to_dict('records')
CLASS_MAP = {}
for i in a:
    CLASS_MAP[i['classe_name']] = i['label_num']
CLASS_MAP

{'Jennie': 0, 'Jisoo': 1, 'Lisa': 2, 'Rose': 3, 'Others': 4}

In [10]:
CLASS_MAP = {'Jennie': 0, 'Jisoo': 1, 'Lisa': 2, 'Others': 4, 'Rose': 3}

In [36]:

# 影像大小
IMAGE_SIZE = (224, 224)
# IMAGE_SIZE = (299, 299)
# 影像類別數
NUM_CLASSES = 5
# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 32
# 凍結網路層數
# FREEZE_LAYERS = 1
# Epoch 數
NUM_EPOCHS = 40
# 模型輸出儲存的檔案
WEIGHTS_FINAL = 'model-resnet50-final.h5'

# 假設您的DataFrame稱為df，包含'file_jpg_path'和'label'欄位
# 您需要將路徑改成絕對路徑或相對於DATASET_PATH的路徑

# train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
#                                    validation_split=0.2)

train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=preprocess_input,  # 將預處理函數設置為 preprocess_input
    validation_split=0.2
)

train_image_gen = train_datagen.flow_from_directory('/kaggle/input/face-recognition-xt122/Train',
                                            target_size=IMAGE_SIZE,
                                            batch_size=BATCH_SIZE,
                                            class_mode='categorical',
                                            classes=CLASS_MAP,
                                            subset='training')

valid_image_gen = train_datagen.flow_from_directory('/kaggle/input/face-recognition-xt122/Train',
                                            target_size=IMAGE_SIZE,
                                            batch_size=BATCH_SIZE,
                                            class_mode='categorical',
                                            classes=CLASS_MAP,
                                            subset='validation')

Found 461 images belonging to 5 classes.
Found 115 images belonging to 5 classes.


In [38]:
base_model = VGG16(
     weights='imagenet',
     include_top=False # 如果是 true 表示要拿這model的classes 來分類
     )

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x) 
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
preds = Dense(5, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=preds)
for layer in model.layers[:-4]:
  layer.trainable=False
       
for layer in model.layers[-4:]:
  layer.trainable=True


In [39]:
model.compile(loss='categorical_crossentropy',
              optimizer='RMSprop',
              metrics=['accuracy'])

In [40]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import os
   
checkpoint = ModelCheckpoint('kaggle-image-classification.hdf5', monitor='val_accuracy', save_best_only=True, mode='auto', verbose=1)
earlystop = EarlyStopping(monitor='val_accuracy', mode='max', patience=15, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2,patience=10, min_lr=0.001)

model_history = model.fit(train_image_gen,
                              epochs=40,
                              steps_per_epoch=(train_image_gen.samples//BATCH_SIZE),
                              validation_data=valid_image_gen,
                              validation_steps=(valid_image_gen.samples//BATCH_SIZE),
                              callbacks=[
                                  checkpoint,
                                  earlystop,
                                  reduce_lr
                              ])

Epoch 1/40
14/14 [==============================] - ETA: 0s - loss: 2.0267 - accuracy: 0.2075
Epoch 1: val_accuracy improved from -inf to 0.20833, saving model to kaggle-image-classification.hdf5
14/14 [==============================] - 14s 857ms/step - loss: 2.0267 - accuracy: 0.2075 - val_loss: 1.6008 - val_accuracy: 0.2083 - lr: 0.0010
Epoch 2/40
14/14 [==============================] - ETA: 0s - loss: 1.6175 - accuracy: 0.2331
Epoch 2: val_accuracy improved from 0.20833 to 0.23958, saving model to kaggle-image-classification.hdf5
14/14 [==============================] - 11s 795ms/step - loss: 1.6175 - accuracy: 0.2331 - val_loss: 1.5975 - val_accuracy: 0.2396 - lr: 0.0010
Epoch 3/40
14/14 [==============================] - ETA: 0s - loss: 1.5911 - accuracy: 0.2611
Epoch 3: val_accuracy improved from 0.23958 to 0.26042, saving model to kaggle-image-classification.hdf5
14/14 [==============================] - 11s 776ms/step - loss: 1.5911 - accuracy: 0.2611 - val_loss: 1.5672 - val_a

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
# from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import MobileNet, MobileNetV2, VGG16, VGG19, Xception,ResNet101V2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras import datasets, layers, Model, Sequential, losses
# 建立類別名稱轉class idx字典
aa = pd.read_csv("/kaggle/input/face-recognition-xt122/ClassMap.csv")
a = aa.to_dict('records')
CLASS_MAP = {}
for i in a:
    CLASS_MAP[i['classe_name']] = i['label_num']
    
NUM_CLASS = len(CLASS_MAP)
CLASS_MAP = {'Jennie': 0, 'Jisoo': 1, 'Lisa': 2, 'Others': 4, 'Rose': 3}
CLASS_MAP, NUM_CLASS

({'Jennie': 0, 'Jisoo': 1, 'Lisa': 2, 'Others': 4, 'Rose': 3}, 5)

In [31]:
import tensorflow as tf
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from sklearn.model_selection import train_test_split

IMG_SIZE = 360
BS = 32

# 撈取有解答之圖片路徑, 分割train, val路徑
paths = glob("/kaggle/input/face-recognition-xt122/Train/*/*.jpg")
train_paths, val_paths = train_test_split(paths,
                                          test_size=0.2,
                                          random_state=5566)

len(train_paths), len(val_paths)

(460, 116)

In [32]:
# 擷取資料夾名稱轉成類別id
# os.sep: "/"
def paths2labels(paths):
    return [CLASS_MAP[p.split(os.sep)[-2]] for p in paths]

# 影像讀取 & resize
def load_and_resize_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    return image

# 使用路徑建構 tf.data.Dataset
def build_ds(paths):
    labels = paths2labels(paths) # paths -> labels
    image_ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    image_ds = image_ds.map(lambda path, label: (load_and_resize_image(path), label)) # path -> img, labels
    return image_ds

# 建構train, val dataset
train_ds = build_ds(train_paths).shuffle(buffer_size=len(train_paths)).batch(BS)
val_ds = build_ds(val_paths).batch(BS)

In [33]:
class WAIModel(tf.keras.models.Model):
    def __init__(self, encoder, preprocess, freeze):
        super().__init__()
        # 前處理函數
        self.preprocess = preprocess
        # 資料擴增
        self.data_aug = tf.keras.Sequential([
            tf.keras.layers.RandomFlip("horizontal"),
            tf.keras.layers.RandomRotation(0.2),
        ])
        # feature extractor
        self.encoder = encoder
        # classifier
        self.classifier = tf.keras.Sequential([
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(NUM_CLASS),
            tf.keras.layers.Softmax(),
        ])
        # freeze: 是否鎖住encoder參數
        if freeze:
            for l in self.encoder.layers:
                l.trainable = False
    def call(self, inputs, training=None):
        x = self.preprocess(inputs)
        # 非訓練時不使用資料擴增
        if training:
            x = self.data_aug(x)

        x = self.encoder(x, training)
        x = self.classifier(x, training)
        return x

In [41]:
tf.keras.backend.clear_session()
# 使用模型對應之前處理函式
preprocess = tf.keras.applications.efficientnet.preprocess_input
encoder = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False,
                                                            weights='imagenet')

for layer in model.layers[:-4]:
    layer.trainable=False
       
for layer in model.layers[-4:]:
    layer.trainable=True
    
model = WAIModel(encoder,
                 preprocess,
                 freeze=False)


In [42]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [43]:
model.encoder.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 rescaling (Rescaling)          (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 normalization (Normalization)  (None, None, None,   7           ['rescaling[0][0]']              
                                3)                                                   

In [44]:
model.fit(train_ds,
          epochs=20,
          validation_data=val_ds,
          callbacks=[
              tf.keras.callbacks.ModelCheckpoint("best.h5",
                                                 save_best_only=True,
                                                 save_weights_only=True)
          ])

Epoch 1/20


2023-08-26 07:09:35.170763: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inwai_model/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


15/15 [==============================] - 53s 871ms/step - loss: 1.3681 - sparse_categorical_accuracy: 0.3957 - val_loss: 2.3401 - val_sparse_categorical_accuracy: 0.2155
Epoch 2/20
15/15 [==============================] - 14s 742ms/step - loss: 0.8701 - sparse_categorical_accuracy: 0.6630 - val_loss: 2.6714 - val_sparse_categorical_accuracy: 0.3534
Epoch 3/20
15/15 [==============================] - 14s 737ms/step - loss: 0.6588 - sparse_categorical_accuracy: 0.7543 - val_loss: 2.6335 - val_sparse_categorical_accuracy: 0.4828
Epoch 4/20
15/15 [==============================] - 14s 746ms/step - loss: 0.4327 - sparse_categorical_accuracy: 0.8587 - val_loss: 1.3466 - val_sparse_categorical_accuracy: 0.5948
Epoch 5/20
15/15 [==============================] - 14s 705ms/step - loss: 0.3044 - sparse_categorical_accuracy: 0.9022 - val_loss: 1.5972 - val_sparse_categorical_accuracy: 0.6034
Epoch 6/20
15/15 [==============================] - 14s 743ms/step - loss: 0.2316 - sparse_categorical_acc

In [45]:
# 讀取最佳val_loss model參數
model.load_weights("/kaggle/working/best.h5")

df = pd.read_csv("/kaggle/input/face-recognition-xt122/SampleSubmission.csv")

# 建構測試dataset
test_paths = [os.path.join(f"/kaggle/input/face-recognition-xt122/Test/{name}.jpg") for name in df["filename"].tolist()]
test_ds = tf.data.Dataset.from_tensor_slices((test_paths))
test_ds = test_ds.map(lambda path: load_and_resize_image(path)).batch(BS)

In [46]:
prediction = model.predict(test_ds)
prediction = np.argmax(prediction, axis=-1)

19/19 [==============================] - 6s 261ms/step


In [47]:
df["label"] = prediction
df.to_csv("submission.csv", index=None)

In [41]:
import os
# from keras.preprocessing import image as k_image
import re
import numpy as np
from keras.models import load_model
import keras.utils as image

r_image = re.compile(r'.*\.(jpg|png|gif)$')
test_image = []
test_image_id = []
for file in os.listdir('/kaggle/input/face-recognition-xt122/Test'):
        img = image.load_img(os.path.join('/kaggle/input/face-recognition-xt122/Test/{}'.format(file)), target_size=(224, 224))
        img = image.img_to_array(img)
        test_image.append(img)
        test_image_id.append(file.split('.')[0])
        
test_train = np.array(test_image)
test_train_id = np.array(test_image_id)
test_train = test_train * 1./255


In [42]:
hdf5_model = load_model('/kaggle/working/kaggle-image-classification.hdf5') 
ans = hdf5_model.predict(test_train)
result = np.argmax(ans,axis=1) # 因為用 Modle 建 model
result = result.reshape((-1, 1))
test_train_id = test_train_id.reshape((-1, 1))

19/19 [==============================] - 3s 145ms/step


In [43]:
result = [str(i).lstrip('[').rstrip(']') for i in result]
# print(test_train_id)
test_train_id = [str(i).replace("'",'').replace("]",'').replace("[",'') for i in test_train_id]
combined_df = pd.DataFrame({'id': test_train_id, 'class': result})

print(combined_df)
# 讀取原始的 CSV 文件
original_df = pd.read_csv('/kaggle/input/face-recognition-xt122/SampleSubmission.csv')

# 合併兩個 DataFrame，根據 id 欄位進行合併
merged_df = pd.merge(original_df, combined_df, left_on='filename', right_on='id', how='left')
# print(merged_df)
# 更新 label 欄位，如果 new_df 中有對應的 label 值就使用它，否則保留原來的值
# merged_df['class'] = merged_df['class'].fillna(merged_df['label'])
merged_df['label'] = merged_df['class']
# 去除多餘的欄位
merged_df = merged_df.drop(['class','id'], axis=1)
print(merged_df)
# 將更新後的 DataFrame 寫回到原始的 CSV 文件
merged_df.to_csv('merged.csv', index=False)

             id class
0    kanbtofrbh     3
1    nzrhioshnt     1
2    molpuqwkks     3
3    opzckherve     0
4    lrwpouhymi     0
..          ...   ...
572  tzncchqcrc     3
573  mnurcrmkkb     0
574  iaamhxjplb     0
575  gfijbhrrzo     2
576  qceiwrvtfs     0

[577 rows x 2 columns]
       filename label
0    wzsoflalzz     3
1    msjwpybvfo     0
2    wfmyruzsyc     1
3    yuobiuggur     0
4    njxpsmkijl     0
..          ...   ...
572  cmnhupznha     1
573  lirjbvolxj     0
574  fjxgoaavtf     0
575  apvlbradzl     3
576  saaqgqcsyk     0

[577 rows x 2 columns]
